* CuDNNLSTM is designed for CUDA parallel processing and cannot run if there is no GPU. But LSTM is designed for normal CPUs. Faster time of execution is because of parallelism.

###### We should use cuDNN

if the model is large.

if the sequence data is long.

if the sequence data has variable lengths.

In [1]:
from tensorflow.keras.datasets import imdb
from tensorflow.python.keras.layers import GRU, LSTM, CuDNNGRU, CuDNNLSTM, Activation
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

##### Import dataset¶

* IMDB dataset in Keras datasets
doc: https://keras.io/datasets/

In [2]:
num_words = 30000
maxlen = 300

In [3]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = num_words)

In [4]:
X_train[:3], y_train[:2]

(array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
        list([1, 194, 115

In [5]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(25000,)
(25000,)
(25000,)
(25000,)


In [6]:
# pad the sequences with zeros 
# padding parameter is set to 'post' => 0's are appended to end of sequences
X_train = pad_sequences(X_train, maxlen = maxlen, padding = 'post')
X_test = pad_sequences(X_test, maxlen = maxlen, padding = 'post')

In [7]:
X_train = X_train.reshape(X_train.shape[0] ,maxlen,1)
X_test = X_test.reshape(X_train.shape[0] ,maxlen,1)

In [8]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(25000, 300, 1)
(25000, 300, 1)
(25000,)
(25000,)


#### Without cudnn

In [9]:
def lstm_model():
    model = Sequential()
    model.add(LSTM(50, input_shape = (300,1), return_sequences = True))
    model.add(LSTM(1, return_sequences = False))
    model.add(Activation('sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [10]:
model = lstm_model()
model

In [14]:
model.fit(X_train, y_train, batch_size = 128, epochs = 10, verbose = 1)

Train on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 137s 5ms/sample - loss: 0.6920 - accuracy: 0.5148
Epoch 2/10
25000/25000 [==============================] - 137s 5ms/sample - loss: 0.6917 - accuracy: 0.5129
Epoch 3/10
25000/25000 [==============================] - 138s 6ms/sample - loss: 0.6915 - accuracy: 0.5141
Epoch 4/10
25000/25000 [==============================] - 133s 5ms/sample - loss: 0.6918 - accuracy: 0.5137
Epoch 5/10
25000/25000 [==============================] - 134s 5ms/sample - loss: 0.6915 - accuracy: 0.5160
Epoch 6/10
25000/25000 [==============================] - 142s 6ms/sample - loss: 0.6913 - accuracy: 0.5167
Epoch 7/10
25000/25000 [==============================] - 137s 5ms/sample - loss: 0.6912 - accuracy: 0.5166
Epoch 8/10
25000/25000 [==============================] - 137s 5ms/sample - loss: 0.6908 - accuracy: 0.5160
Epoch 9/10
25000/25000 [==============================] - 138s 6ms/sample - loss: 0.6906 - accuracy: 0.5192
Epoch

In [22]:
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

25000/25000 [==============================] - 98s 4ms/sample - loss: 0.6931 - accuracy: 0.5043
Accuracy: 50.43%


#### Naive GRU model without CuDNN

In [19]:
def gru_model():
    model = Sequential()
    model.add(GRU(50, input_shape = (300,1), return_sequences = True))
    model.add(GRU(1, return_sequences = False))
    model.add(Activation('sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [20]:
model = gru_model()

In [23]:
model.fit(X_train, y_train, batch_size = 100, epochs = 10, verbose = 1)

Train on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 123s 5ms/sample - loss: 0.6930 - accuracy: 0.5071
Epoch 2/10
25000/25000 [==============================] - 124s 5ms/sample - loss: 0.6923 - accuracy: 0.5109
Epoch 3/10
25000/25000 [==============================] - 125s 5ms/sample - loss: 0.6918 - accuracy: 0.5123
Epoch 4/10
25000/25000 [==============================] - 127s 5ms/sample - loss: 0.6916 - accuracy: 0.5153
Epoch 5/10
25000/25000 [==============================] - 122s 5ms/sample - loss: 0.6907 - accuracy: 0.5180
Epoch 6/10
25000/25000 [==============================] - 123s 5ms/sample - loss: 0.6909 - accuracy: 0.5144
Epoch 7/10
25000/25000 [==============================] - 122s 5ms/sample - loss: 0.6899 - accuracy: 0.5176
Epoch 8/10
25000/25000 [==============================] - 119s 5ms/sample - loss: 0.6891 - accuracy: 0.5221
Epoch 9/10
25000/25000 [==============================] - 119s 5ms/sample - loss: 0.6885 - accuracy: 0.5178
Epoch

In [24]:
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

25000/25000 [==============================] - 99s 4ms/sample - loss: 0.6939 - accuracy: 0.5050
Accuracy: 50.50%


#### CuDNN LSTM

In [25]:
def cudnn_lstm_model():
    model = Sequential()
    model.add(CuDNNLSTM(50, input_shape = (300,1), return_sequences = True))
    model.add(CuDNNLSTM(1, return_sequences = False))
    model.add(Activation('sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [26]:
model = cudnn_lstm_model()

In [27]:
model.fit(X_train, y_train, batch_size = 100, epochs = 10, verbose = 1)

Train on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 8s 339us/sample - loss: 0.6932 - accuracy: 0.5084
Epoch 2/10
25000/25000 [==============================] - 4s 165us/sample - loss: 0.6926 - accuracy: 0.5074 - loss: 0.6928 - ac
Epoch 3/10
25000/25000 [==============================] - 4s 164us/sample - loss: 0.6923 - accuracy: 0.5103 - loss: 0
Epoch 4/10
25000/25000 [==============================] - 4s 165us/sample - loss: 0.6914 - accuracy: 0.5149
Epoch 5/10
25000/25000 [==============================] - 4s 165us/sample - loss: 0.6903 - accuracy: 0.5144
Epoch 6/10
25000/25000 [==============================] - 4s 164us/sample - loss: 0.6900 - accuracy: 0.5132 - loss: 0.6898 - ac
Epoch 7/10
25000/25000 [==============================] - 4s 165us/sample - loss: 0.6901 - accuracy: 0.5202
Epoch 8/10
25000/25000 [==============================] - 4s 165us/sample - loss: 0.6890 - accuracy: 0.5212
Epoch 9/10
25000/25000 [==============================] - 4s 16

In [28]:
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

25000/25000 [==============================] - 14s 546us/sample - loss: 0.6899 - accuracy: 0.5161
Accuracy: 51.61%


#### CuDNNGRU

In [29]:
def cudnn_gru_model():
    model = Sequential()
    model.add(CuDNNGRU(50, input_shape = (300,1), return_sequences = True))
    model.add(CuDNNGRU(1, return_sequences = False))
    model.add(Activation('sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model


In [30]:
model = cudnn_gru_model()

In [31]:
model.fit(X_train, y_train, batch_size = 100, epochs = 10, verbose = 1)

Train on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 5s 187us/sample - loss: 0.6937 - accuracy: 0.5034
Epoch 2/10
25000/25000 [==============================] - 4s 155us/sample - loss: 0.6928 - accuracy: 0.5062
Epoch 3/10
25000/25000 [==============================] - 4s 155us/sample - loss: 0.6919 - accuracy: 0.5074TA: 0s - loss: 0.6919 - accuracy
Epoch 4/10
25000/25000 [==============================] - 4s 155us/sample - loss: 0.6911 - accuracy: 0.5178 - los - ETA: 1s - loss: - ETA: 0s - loss: 0.6
Epoch 5/10
25000/25000 [==============================] - 4s 156us/sample - loss: 0.6901 - accuracy: 0.5183 - loss: 0.6902 - accura
Epoch 6/10
25000/25000 [==============================] - 4s 155us/sample - loss: 0.6898 - accuracy: 0.5210 ETA: 0s -
Epoch 7/10
25000/25000 [==============================] - 4s 155us/sample - loss: 0.6891 - accuracy: 0.5215
Epoch 8/10
25000/25000 [==============================] - 4s 157us/sample - loss: 0.6885 - accuracy: 0.5190 -

In [32]:
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

25000/25000 [==============================] - 14s 543us/sample - loss: 0.6879 - accuracy: 0.5191
Accuracy: 51.91%
